# Credit: [Rongyao Fang](https://github.com/rongyaofang)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from sklearn.metrics import roc_auc_score
# from tensorboardX import SummaryWriter
import tensorboard_logger as tb_logger

In [2]:
PARAMS = {
    "input_channels": 1,
    "output_channels": 2,
    "input_transform_fn": lambda x: x / 128. - 1.,
    "input_conv_channels": 32,
#     'down_structure': [2,2,2],
    'down_structure': [3, 8, 4],
    'activation_fn': lambda: nn.LeakyReLU(0.1, inplace=True),
    # "normalization_fn": lambda c: nn.GroupNorm(num_groups=4, num_channels=c),
    "normalization_fn": lambda c: nn.BatchNorm3d(num_features=c),
    "drop_rate": 0,
    'growth_rate': 32,
    'bottleneck': 4,
    'compression': 2,
    'use_memonger': False  # ~2.2x memory efficiency (batch_size), 25%~30% slower on GTX 1080.
}

In [3]:
def densenet3d(with_segment=False, snapshot=None, **kwargs):
    for k, v in kwargs.items():
        assert k in PARAMS
        PARAMS[k] = v
    print("Model hyper-parameters:", PARAMS)
    if with_segment:
        model = DenseSharp()
        print("Using DenseSharp model.")
    else:
        model = DenseNet()
        print("Using DenseNet model.")
    print(model)
    if snapshot is None:
        initialize(model.modules())
        print("Random initialized.")
    else:
        state_dict = torch.load(snapshot)
        model.load_state_dict(state_dict)
        print("Load weights from `%s`," % snapshot)
    return model

In [4]:
def initialize(modules):
    for m in modules:
        if isinstance(m, nn.Conv3d) or isinstance(m, nn.ConvTranspose3d):
            nn.init.kaiming_uniform_(m.weight, mode='fan_in')
        elif isinstance(m, nn.Linear):
            nn.init.kaiming_uniform_(m.weight, mode='fan_in')
            m.bias.data.zero_()

In [5]:
class ConvBlock(nn.Sequential):
    def __init__(self, in_channels):
        super(ConvBlock, self).__init__()

        growth_rate = PARAMS['growth_rate']
        bottleneck = PARAMS['bottleneck']
        activation_fn = PARAMS['activation_fn']
        normalization_fn = PARAMS['normalization_fn']

        self.in_channels = in_channels
        self.growth_rate = growth_rate
        self.use_memonger = PARAMS['use_memonger']
        self.drop_rate = PARAMS['drop_rate']

        # TODO: consider bias term in conv with GN
        self.add_module('norm_1', normalization_fn(in_channels))
        self.add_module('act_1', activation_fn())
        self.add_module('conv_1', nn.Conv3d(in_channels, bottleneck * growth_rate, kernel_size=1, stride=1,
                                            padding=0, bias=True))

        self.add_module('norm_2', normalization_fn(bottleneck * growth_rate))
        self.add_module('act_2', activation_fn())
        self.add_module('conv_2', nn.Conv3d(bottleneck * growth_rate, growth_rate, kernel_size=3, stride=1,
                                            padding=1, bias=True))

    def forward(self, x):
        super_forward = super(ConvBlock, self).forward
        if self.use_memonger:
            new_features = checkpoint(super_forward, x)
        else:
            new_features = super_forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)

    @property
    def out_channels(self):
        return self.in_channels + self.growth_rate

In [6]:
class TransmitBlock(nn.Sequential):
    def __init__(self, in_channels, is_last_block):
        super(TransmitBlock, self).__init__()

        activation_fn = PARAMS['activation_fn']
        normalization_fn = PARAMS['normalization_fn']
        compression = PARAMS['compression']

        # print("in_channels: %s, compression: %s" % (in_channels, compression))
        assert in_channels % compression == 0

        self.in_channels = in_channels
        self.compression = compression

        self.add_module('norm', normalization_fn(in_channels))
        self.add_module('act', activation_fn())
        if not is_last_block:
            self.add_module('conv', nn.Conv3d(in_channels, in_channels // compression, kernel_size=(1, 1, 1),
                                              stride=1, padding=0, bias=True))
            self.add_module('pool', nn.AvgPool3d(kernel_size=2, stride=2, padding=0))
        else:
            self.compression = 1

    @property
    def out_channels(self):
        return self.in_channels // self.compression

In [7]:
class Lambda(nn.Module):
    def __init__(self, lambda_fn):
        super(Lambda, self).__init__()
        self.lambda_fn = lambda_fn

    def forward(self, x):
        return self.lambda_fn(x)

In [8]:
class DenseNet(nn.Module):

    def __init__(self):

        super(DenseNet, self).__init__()

        input_channels = PARAMS['input_channels']
        input_transform_fn = PARAMS['input_transform_fn']
        input_conv_channels = PARAMS['input_conv_channels']
        normalization_fn = PARAMS['normalization_fn']
        activation_fn = PARAMS['activation_fn']
        down_structure = PARAMS['down_structure']
        output_channels = PARAMS['output_channels']

        self.features = nn.Sequential()
        if input_transform_fn is not None:
            self.features.add_module("input_transform", Lambda(input_transform_fn))
        self.features.add_module("init_conv", nn.Conv3d(input_channels, input_conv_channels, kernel_size=3,
                                                        stride=1, padding=1, bias=True))
        self.features.add_module("init_norm", normalization_fn(input_conv_channels))
        self.features.add_module("init_act", activation_fn())

        channels = input_conv_channels
        for i, num_layers in enumerate(down_structure):
            for j in range(num_layers):
                conv_layer = ConvBlock(channels)
                self.features.add_module('denseblock{}_layer{}'.format(i + 1, j + 1), conv_layer)
                channels = conv_layer.out_channels
                # print(i, j, channels)

            trans_layer = TransmitBlock(channels, is_last_block=i == len(down_structure) - 1)
            self.features.add_module('transition%d' % (i + 1), trans_layer)
            channels = trans_layer.out_channels

        self.classifier = nn.Linear(channels, output_channels)

    def forward(self, x, **return_opts):
        # o = x
        # for i, layer in enumerate(self.features):
        #     o = layer(o)
        #     print(i, layer, o.shape)

        batch_size, _, d, h, w = x.size()

        features = self.features(x)
        pooled = F.adaptive_avg_pool3d(features, 1).view(batch_size, -1)
        scores = self.classifier(pooled)

        if len(return_opts) == 0:
            return scores

        # return other features in one forward
        for opt in return_opts:
            assert opt in {"return_features", "return_cam"}
        # print(return_opts)

        ret = dict(scores=scores)

        if 'return_features' in return_opts and return_opts['return_features']:
            ret['features'] = features

        if 'return_cam' in return_opts and return_opts['return_cam']:
            weight = self.classifier.weight.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
            bias = self.classifier.bias
            cam_raw = F.conv3d(features, weight, bias)
            cam = F.interpolate(cam_raw, size=(d, h, w), mode='trilinear', align_corners=True)
            ret['cam'] = F.softmax(cam, dim=1)
            ret['cam_raw'] = F.softmax(cam_raw, dim=1)
        return ret

In [9]:
from torch.utils.data import Dataset
import random
import torch
import os
import torch.nn as nn
from tqdm import tqdm

import pandas as pd

import numpy as np

class ClfDataset(Dataset):

    def __init__(self, train=True):
        self.train = train
        data_dir = './dataset/'
        # choose the dataset
        patients_train = os.listdir(data_dir+'train_val/')
        patients_train.sort(key= lambda x:int(x[9:-4]))
        patients_test = os.listdir(data_dir+'test/')
        patients_test.sort(key= lambda x:int(x[9:-4]))
        labels_df = pd.read_csv('./dataset/info.csv',index_col=0)

        self.data_train = []
        self.data_test = []
        self.labels = []
        self.names_train = []
        self.names_test = []

        for num_train, patient_train in enumerate(patients_train):
            patient_train_name = patient_train[0:-4]
            self.names_train.append(patient_train_name)
            label = labels_df.get_value(patient_train_name, 'lable')
            path_train = data_dir + 'train_val/' + patient_train
            img_data_train = np.load(path_train)
            voxel_train = img_data_train['voxel'].astype(np.int32)
            voxel_train_crop = voxel_train[10:90,10:90,10:90]
            self.data_train.append(voxel_train_crop)
            self.labels.append(label)

        for num_test, patient_test in enumerate(patients_test):
            self.names_test.append(patient_test[0:-4])
            path_test = data_dir + 'test/' + patient_test
            img_data_test = np.load(path_test)
            voxel_test = img_data_test['voxel'].astype(np.int32)
            voxel_test_crop = voxel_test[10:90,10:90,10:90]
            self.data_test.append(voxel_test_crop)
    
    def __getitem__(self, item):
        if self.train:
            patient_data_train = self.data_train[item]
            patient_label = self.labels[item]
            patient_name_train = self.names_train[item]
            return patient_data_train, patient_label, patient_name_train
        else:
            patient_data_test = self.data_test[item]
            patient_name_test = self.names_test[item]
            return patient_data_test, patient_name_test
        
    def __len__(self):
        if self.train:
            return len(self.labels)
        else:
            return len(self.data_test)

In [10]:
# '''[begin] load the train and validation dataset:'''
# import numpy as np
# # use np.load to import data, devide dataset into 2 parts: train_data & validation_data:
# # credit: cheez & Matthew Kerian
# # link: https://stackoverflow.com/questions/55890813/how-to-fix-object-arrays-cannot-be-loaded-when-allow-pickle-false-for-imdb-loa/56243777
# '''use older to successfully load the data:'''
# # save np.load
# np_load_old = np.load

# # modify the default parameters of np.load
# np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# data_ineed = np.load('muchdata.npy')

# # restore np.load for future normal usage
# np.load = np_load_old

# # If you are working with the basic sample data, use maybe 2 instead of 100 here... you don't have enough data to really do this
# train_data = data_ineed[:-60]
# validation_data = data_ineed[-60:]
# '''[end] load the train and validation dataset:'''

model = densenet3d(with_segment=False, use_memonger=True).cuda()
model = nn.DataParallel(model, device_ids=[0, 1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# optimizer = optim.SGD(model.parameters(),lr=0.001, momentum=0.9)

criterion = criterion.cuda()

from torch.utils.data import DataLoader

data_train = ClfDataset(train=True)
train_data_loader = DataLoader(dataset=data_train, batch_size=64, shuffle=False)
dev_data_loader = DataLoader(dataset=data_train, batch_size=64, shuffle=False)
data_test = ClfDataset(train=False)
test_data_loader = DataLoader(dataset=data_test, batch_size=64, shuffle=False)

Model hyper-parameters: {'input_channels': 1, 'output_channels': 2, 'input_transform_fn': <function <lambda> at 0x7fb4779e6620>, 'input_conv_channels': 32, 'down_structure': [3, 8, 4], 'activation_fn': <function <lambda> at 0x7fb4779e66a8>, 'normalization_fn': <function <lambda> at 0x7fb4779e6730>, 'drop_rate': 0, 'growth_rate': 32, 'bottleneck': 4, 'compression': 2, 'use_memonger': True}
Using DenseNet model.
DenseNet(
  (features): Sequential(
    (input_transform): Lambda()
    (init_conv): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (init_norm): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (init_act): LeakyReLU(negative_slope=0.1, inplace=True)
    (denseblock1_layer1): ConvBlock(
      (norm_1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_1): LeakyReLU(negative_slope=0.1, inplace=True)
      (conv_1): Conv3d(32, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1))
     

/home/vivi/anaconda3/envs/3dunet/lib/python3.7/site-packages/ipykernel_launcher.py:33: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [11]:
data_test[0][1]

'candidate11'

### Train & Valiation:

In [ ]:
date = 24 # day
number = 8 # No.
sample_size = 32
file_name = 'train_{}_{}'.format(date,number)
# writer = SummaryWriter('./runs/{}'.format(file_name))
logger = tb_logger.Logger(logdir = './runs/train_{}_{}'.format(date,number), flush_secs=2)
# val_logger = tb_logger.Logger(logdir = './runs/train_{}_{}/test'.format(date,number), flush_secs=2)

names_box_train = list()
labels_box_train = []

for epoch in range(200):  # loop over the dataset multiple times
    
    # for train:
    running_loss = 0.0
    # for valiation:
    dev_loss = 0
    predict_value = torch.zeros(1) # valiation predict:
    true_value = torch.zeros(1) # true lable of valiation part:
    
    for index, (inputs, labels, patient_name) in enumerate(tqdm(train_data_loader)):
        
        names_box_train.extend(patient_name)
        labels_box_train.extend(labels.cpu().numpy().tolist())
        
        if index<=5:# train part: 
            
            model.train()
            
            # zero the parameter gradients
            optimizer.zero_grad()
        
            inputs, labels = inputs.cuda(), labels.cuda()
        
            # forward + backward + optimize
            inputs = inputs.unsqueeze(dim=1).float()
        
            inputs = F.interpolate(inputs, size=[sample_size,sample_size,sample_size],mode='trilinear',align_corners=False)
        
            outputs = model(inputs)
        
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
            # print statistics
            running_loss += loss.item()
            
        else: # valiation part:
            
            model.eval()
            
            inputs, labels = inputs.cuda(), labels.cuda()
        
            # forward + backward + optimize
            inputs = inputs.unsqueeze(dim=1).float()
        
            inputs = F.interpolate(inputs, size=[sample_size,sample_size,sample_size],mode='trilinear',align_corners=False)
        
            outputs = model(inputs)
        
            test_value = F.softmax(outputs)
            test_value_1 = test_value[:,1]
            test_value_1 = test_value_1.cpu()
            predict_value = torch.cat((predict_value,test_value_1))
    
            test_value_2 = labels
            test_value_2 = test_value_2.cpu().float()
            true_value = torch.cat((true_value, test_value_2))
            
    print('[%d] loss: %.5f' %(epoch + 1, running_loss / 384))
#     writer.add_scalar('Loss', running_loss, epoch)
    logger.log_value('loss',running_loss,epoch)
    running_loss = 0.0
        
    predict_value = predict_value.detach().numpy()
    true_value = true_value.numpy()
    auc_score = roc_auc_score(true_value, predict_value)
    print(auc_score)
#     writer.add_scalar('Auc', auc_score, epoch)
    logger.log_value('AUC',auc_score, epoch)
    
# writer.close()

  0%|          | 0/8 [00:00<?, ?it/s]

[1] loss: 0.01223
0.6005952380952381


  0%|          | 0/8 [00:00<?, ?it/s]

[2] loss: 0.01157
0.6029761904761904


  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

[150] loss: 0.00000
0.5428571428571428


  0%|          | 0/8 [00:00<?, ?it/s]

[151] loss: 0.00000
0.5422619047619047


 62%|██████▎   | 5/8 [00:05<00:03,  1.11s/it]

In [ ]:
# import csv
# box_train = zip(names_box_train, labels_box_train)
# # print(box_train)
# with open('output.csv','w') as result_file:
#     wr = csv.writer(result_file, dialect='excel')
#     for row in box_train:
#         wr.writerow(row)

In [ ]:
PATH = './my_{}_{}.pth'.format(date,number)
torch.save(model.state_dict(), PATH)

In [ ]:
# ./my_23_1.pth 100epochs
# my_24_1.pth 300epochs train&dev Adam 0.62(200) batch_size = 32
# my_24_2.pth 200epochs train&dev SGD 0.656(200) batch_size = 32
# my_24_3.pth 200epochs train&dev SGD 0.633(200) batch_size = 16
# my_24_4.pth 200epochs train&dev SGD 0.625(200) batch_size = 32 crop 80-> 32
# my_24_5.pth 200epochs train&dev Adam 0.600(200) batch_size = 32 crop 80-> 32 0.689(20)
# my_24_6.pth  25epochs train&dev Adam           batch_size = 32 crop 80-> 32  0.689(25)
# my_24_7.pth  25epochs train     Adam           batch_size = 32 crop 80-> 32             test:  0.5
# my_24_8.pth 200epochs train&dev Adam           batch_size = 64 crop 80-> 32             notes: 不收敛 下一步调小batch，增加采样点数


In [ ]:
# date = 24 # day
# number = 7 # No.
# PATH = './my_{}_{}.pth'.format(date,number)
# model = densenet3d(with_segment=False, use_memonger=True).cuda()
# model.load_state_dict(torch.load(PATH))

model.eval()
# dev_loss = 0
predict_value = []
# predict_value = np.array(predict_value)
# predict_value = torch.from_numpy(predict_value)
names_box_test = []

for index, (inputs,patient_name) in enumerate(tqdm(test_data_loader)):
    
    inputs = inputs.cuda()
        # forward + backward + optimize
    inputs = inputs.unsqueeze(dim=1).float()
        
    inputs = F.interpolate(inputs, size=[32,32,32],mode='trilinear',align_corners=False)
        
    outputs = model(inputs)
    
    test_value = F.softmax(outputs)
    test_value_1 = test_value[:,1]
    test_value_1 = test_value_1.cpu()
    predict_value.extend(test_value_1.detach().numpy().tolist())
#     print(predict_value)
    
#     predict_value = torch.cat((predict_value,test_value_1))
    names_box_test.extend(patient_name)
    
#     if index==1:
#         break
    
#     test_value_2 = labels
#     test_value_2 = test_value_2.cpu().float()
#     true_value = torch.cat((true_value, test_value_2))


# test_predict_value = predict_value.detach().numpy()



In [ ]:
import csv
box_train = zip(names_box_test, predict_value)
# print(box_train)
with open('output_{}_{}.csv'.format(date,number),'w') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    for row in box_train:
        wr.writerow(row)
        
print('---------Success, please check output_{}_{}.csv---------'.format(date,number))

In [ ]:
# for epoch in range(4):  # loop over the dataset multiple times
#     running_loss = 0.0
#     for inputs, labels in train_data_loader:
#         # get the inputs; data is a list of [inputs, labels]
#         # inputs: tensor(100*(100*100)), label
#         inputs_numpy, labels = data
#         inputs = torch.from_numpy(inputs_numpy)
#         inputs = inputs.unsqueeze(0)
#         inputs = inputs.unsqueeze(0)
#         inputs = inputs.cuda()
# #         inputs = inputs.float()
#         labels = torch.from_numpy(np.asarray(labels)).float()
#         labels = labels.long() #credit: ptrblck; link: https://discuss.pytorch.org/t/runtimeerror-expected-object-of-scalar-type-long-but-got-scalar-type-float-when-using-crossentropyloss/30542/4
#         labels = labels.unsqueeze(0)

#         labels = labels.cuda()
#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = model(inputs)
        
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 20 == 19:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 20))
#             running_loss = 0.0

# print('Finished Training')

### Only train:

In [ ]:
# for epoch in range(100):  # loop over the dataset multiple times
    
#     model.train()
#     running_loss = 0.0
#     for index, (inputs, labels) in enumerate(tqdm(train_data_loader)):
#         # zero the parameter gradients
#         optimizer.zero_grad()
        
#         inputs, labels = inputs.cuda(), labels.cuda()
        
#         # forward + backward + optimize
#         inputs = inputs.unsqueeze(dim=1).float()
        
#         inputs = F.interpolate(inputs, size=[32,32,32],mode='trilinear',align_corners=False)
        
#         outputs = model(inputs)
        
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
        
#         # print statistics
#         running_loss += loss.item()
        
#         if index==11:
#             break
        

#     print('[%d] loss: %.5f' %
#         (epoch + 1, running_loss / 352))
#     running_loss = 0.0

### Only Valiation:

In [ ]:
# PATH = './my_23_1.pth'
# model = densenet3d(with_segment=False, use_memonger=True).cuda()
# model.load_state_dict(torch.load(PATH))

# model.eval()
# dev_loss = 0
# predict_value = torch.zeros(1)
# true_value = torch.zeros(1)

# for index, (inputs, labels) in enumerate(tqdm(train_data_loader)):
#     if index<=11:
#         continue

#     inputs, labels = inputs.cuda(), labels.cuda()
        
#         # forward + backward + optimize
#     inputs = inputs.unsqueeze(dim=1).float()
        
#     inputs = F.interpolate(inputs, size=[32,32,32],mode='trilinear',align_corners=False)
        
#     outputs = model(inputs)
    
#     test_value = F.softmax(outputs)
#     test_value_1 = test_value[:,1]
#     test_value_1 = test_value_1.cpu()
#     predict_value = torch.cat((predict_value,test_value_1))
    
#     test_value_2 = labels
#     test_value_2 = test_value_2.cpu().float()
#     true_value = torch.cat((true_value, test_value_2))
# #     loss = criterion(outputs, labels)    
# #     dev_loss+=loss.item()
    
# # dev_loss = dev_loss/64
# # print("%.4f" %dev_loss)

# predict_value = predict_value.detach().numpy()
# true_value = true_value.numpy()

# from sklearn.metrics import roc_auc_score
# roc_auc_score(true_value, predict_value)

In [ ]:
# 验证加入名字之后是否对应

## Test Part

In [ ]:
# # date = 24 # day
# # number = 7 # No.
# # PATH = './my_{}_{}.pth'.format(date,number)
# # model = densenet3d(with_segment=False, use_memonger=True).cuda()
# # model.load_state_dict(torch.load(PATH))

# model.eval()
# # dev_loss = 0
# predict_value = torch.zeros(1)
# # true_value = torch.zeros(1)

# for _, inputs in enumerate(tqdm(test_data_loader)):
    
#     inputs = inputs.cuda()
#         # forward + backward + optimize
#     inputs = inputs.unsqueeze(dim=1).float()
        
#     inputs = F.interpolate(inputs, size=[32,32,32],mode='trilinear',align_corners=False)
        
#     outputs = model(inputs)
    
#     test_value = F.softmax(outputs)
#     test_value_1 = test_value[:,1]
#     test_value_1 = test_value_1.cpu()
#     print(test_value_1)
    
#     predict_value = torch.cat((predict_value,test_value_1))
    
# #     test_value_2 = labels
# #     test_value_2 = test_value_2.cpu().float()
# #     true_value = torch.cat((true_value, test_value_2))


# test_predict_value = predict_value.detach().numpy()


In [ ]:
# np.savetxt('new_{}_{}.csv'.format(date,number), test_predict_value, delimiter = ',')